In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#optimization: AdamW
#model: microsoft/CodeBERT
#learning rate: 1e-5
#train_test_split=0.1, seed = 100
#batch_size = 16

In [3]:
!pip install torch
!pip install transformers
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 6.6 MB 41.9 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.6 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.4 MB/s 
     |████████████████████████████████| 140 kB 49.0 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████

In [4]:
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer, RobertaConfig, RobertaModel
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss

In [5]:
#Load Train dataset
train = pd.read_csv('new_train_sample.csv', encoding = 'utf-8')
df = pd.DataFrame(train)

In [6]:
#Define Model
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#tokenizer.truncation_side = 'left'

batch_size = 16
epoch_num = 10
MAX_LEN = 512
learning_rate = 1e-5

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be 

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [7]:
def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [8]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')
encoded_dataset
print(encoded_dataset[1])
print(len(encoded_dataset))
print(tokenizer.tokenize(dataset['code2'][1]))

Using custom data configuration default-9ba85ce7a16169d0


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9ba85ce7a16169d0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

{'labels': 1, 'input_ids': [0, 250, 5214, 2544, 1640, 46797, 49338, 50118, 387, 5214, 2544, 1640, 46797, 49338, 50118, 1594, 112, 45, 11, 646, 250, 6, 387, 42645, 50118, 50117, 17265, 1640, 134, 43, 50118, 1594, 132, 45, 11, 646, 250, 6, 387, 42645, 50118, 50117, 17265, 1640, 176, 43, 50118, 1594, 155, 45, 11, 646, 250, 6, 387, 42645, 50118, 50117, 17265, 1640, 246, 43, 2, 2, 257, 49248, 134, 6, 176, 6, 246, 24303, 50118, 4546, 49310, 2544, 1640, 46797, 49338, 13, 939, 11, 1186, 1640, 176, 46077, 50118, 17265, 1640, 8458, 1640, 257, 12, 8738, 1640, 4546, 35122, 10975, 288, 45587, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

In [10]:
from transformers import AdamW
import transformers
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred): #이게 필요한가?
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [13]:
'''
class TrainerModified(Trainer):
  def compute_loss(self,model,inputs,return_outputs=False):
    labels = inputs.get("labels")
    outputs = model()
'''

'\nclass TrainerModified(Trainer):\n  def compute_loss(self,model,inputs,return_outputs=False):\n    labels = inputs.get("labels")\n    outputs = model()\n'

In [14]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


In [15]:
gc.collect()
torch.cuda.empty_cache()

In [16]:
trainer.train()

***** Running training *****
  Num examples = 36000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 22500


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342900,0.188038,0.932250
2,0.182300,0.175586,0.947250
3,0.151900,0.175263,0.953250
4,0.125400,0.148796,0.960250
5,0.104100,0.178373,0.964250
6,0.086500,0.203258,0.961750
7,0.071700,0.189185,0.966000
8,0.059800,0.192228,0.968750
9,0.050700,0.217930,0.967000
10,0.042000,0.242158,0.966500


***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16
Saving model checkpoint to test/checkpoint-2250
Configuration saved in test/checkpoint-2250/config.json
Model weights saved in test/checkpoint-2250/pytorch_model.bin
tokenizer config file saved in test/checkpoint-2250/tokenizer_config.json
Special tokens file saved in test/checkpoint-2250/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16
Saving model checkpoint to test/checkpoint-4500
Configuration saved in test/checkpoint-4500/config.json
Model weights saved in test/checkpoint-4500/pytorch_model.bin
tokenizer config file saved in test/checkpoint-4500/tokenizer_config.json
Special tokens file saved in test/checkpoint-4500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16
Saving model checkpoint to test/checkpoint-6750
Configuration saved in test/checkpoint-6750/config.json
Model weights saved in test/checkpoint-6750/pytorch_mod

TrainOutput(global_step=22500, training_loss=0.12173500569661458, metrics={'train_runtime': 19927.421, 'train_samples_per_second': 18.066, 'train_steps_per_second': 1.129, 'total_flos': 9.47199799296e+16, 'train_loss': 0.12173500569661458, 'epoch': 10.0})

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.96875,
 'eval_loss': 0.19222785532474518,
 'eval_runtime': 70.5708,
 'eval_samples_per_second': 56.681,
 'eval_steps_per_second': 3.543}

In [18]:
#test data preprocess 추가
TEST = "test.csv"
SAMPLE = "sample_submission.csv"

test = pd.read_csv('test.csv', encoding = 'utf-8')
df_test = pd.DataFrame(test)

preprocess(df_test,"preprocess_test.csv")

test_dataset = load_dataset("csv",data_files = "preprocess_test.csv")['train']
test_dataset = test_dataset.map(tokenized,remove_columns=['code1','code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SAMPLE)
df['similar'] = np.argmax(predictions.predictions,axis=-1)
df.to_csv('/content/drive/MyDrive/submission.csv',index=False)

Using custom data configuration default-bb3b4321c6118c1b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bb3b4321c6118c1b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: pair_id, Unnamed: 0. If pair_id, Unnamed: 0 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 16
